In [1]:
import numpy
import pymongo 
from sklearn.feature_extraction import DictVectorizer 
import h2o 
import pandas 
import re 
import random 

In [2]:
def clean_record(json_record):
    record = json.loads(json_record)
    return pandas.io.json.json_normalize(record)

def flatten_record(json_record):
    record = json.loads(json_record)
    facilities = pandas.io.json.json_normalize(record['facilities'])
    number_of_rooms = record['number_of_rooms']
    return facilities

def json_flatten(json_record, sep='::'):

    def flatten(record, node=''):
        new_record = {}
        if type(record) is dict:
            for k, v in record.items():
                if node is not '':
                    new_record.update(flatten(v, node + sep + k))
                else:
                    new_record.update(flatten(v, node + k)) 
        else:
            new_record.update({node : record })
        return new_record

    record = flatten(json_record)

    return record


def remove_empty_nodes(record):
    new_record = record.copy()
    for k, v in record.items():
        if type(v) is list:
            if len(v) is 0:
                del new_record[k]
        elif type(v) is str:
            if len(v.strip()) is 0:
                del new_record[k]
    return new_record

def remove_nodes(record, nodes=[]):
    new_record = record.copy()
    for node in nodes:
        if node in new_record:
            del new_record[node]
    return new_record

def list_to_features(list_feature): 
    if type(list_feature) is list:
        return {value.strip(): 1 for value in list_feature} 
    
def string_to_features(string_feature, sep=','):
    if type(string_feature) is str:
        return {value.strip(): 1 for value in string_feature.split(sep) if len(value.strip()) is not 0}
    
def normalize_features(features): 
    return [re.sub('[^a-zA-Z0-9_.]', '', feature.strip().replace(" ", "_").replace("/", "_").replace("-", "_").replace(">", "greater").replace("<", "lower")) for feature in features]

def rooms_features(record): 
    new_records = []
    for room in record['rooms']: 
        facilities = room['facilities']
        size_string = re.findall('\d*\.?\d*[ ]* m²', facilities)
        if len(size_string) > 0: 
            size = float(re.findall('\d*\.?\d*', size_string[0])[0])
        else: 
            size = 0 
        facilities = re.sub('\d*\.?\d*[ ]* m²', '', facilities)
        new_record = string_to_features(facilities) 
        if size > 0: 
            new_record.update({'Size': size})
        name = room['name']
        new_records.append(new_record) 
    return new_records 

def prepare_record(record):
    new_record = record.copy()
    for v in new_record['facilities']:
        new_record['facilities'][v] = list_to_features(record['facilities'][v])
    if len(record['surroundings'].strip()) is not 0:
        new_record['surroundings'] = string_to_features(record['surroundings'])
    else:
        del new_record['surroundings'] 
    new_record = remove_nodes(new_record, ['_id', 'rooms', 'description', 'reviews', 'url', 'updated', 'chain', 'popular_facilities', 'title', 'highlights', 'check_out', 'check_in', 'hotel_id'])
    new_record = json_flatten(new_record, '.')
    new_record = remove_empty_nodes(new_record)
    if 'number_of_rooms' in new_record: 
        #new_record['number_of_rooms'] = int(new_record['number_of_rooms']) 
        del new_record['number_of_rooms'] 
    return { re.sub('[^a-zA-Z0-9_.]', '', k.replace(" ", "_").replace("/", "_").replace("-", "_").replace(">", "greater").replace("<", "lower")): v for k, v in new_record.items()}

def mongo_collection(host='localhost', port=27017, db='booking', collection='hotel_details'):
    client = pymongo.MongoClient(host, port)
    return client[db][collection]

def rooms_records(records): 
    new_records = []
    y = []
    for record in records:
        for new_record in rooms_features(record): 
            new_records.append(new_record)
            y.append(record['type'])  
    return new_records, y 

def split_records(records, ratio): 
    train = [] 
    valid = []
    for record in records: 
        if random.random() <= ratio: 
            train.append(record) 
        else: 
            valid.append(record)    
    return train, valid 

In [3]:
collection = mongo_collection() 
records = collection.find({'$or': [{'type': 'apartments'}, {'type': 'hotel'}]})

In [4]:
room_features, y = rooms_records(records) #Extract room features and types from property records 

In [5]:
vectorizer = DictVectorizer() 
X = vectorizer.fit_transform(room_features)

In [6]:
Xy = pandas.DataFrame(X.toarray(), columns=normalize_features(vectorizer.get_feature_names())) 
Xy['type'] = y  

In [7]:
h2o.init()

Connecting to H2O server at http://localhost:54321... successful!


H2O cluster uptime:,14 hours 6 mins
H2O cluster version:,3.10.0.3
H2O cluster version age:,18 days
H2O cluster name:,H2O_from_python_I_eu98dh
H2O cluster total nodes:,1
H2O cluster free memory:,3.133 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster is healthy:,True
H2O cluster is locked:,True
H2O connection url:,http://localhost:54321


In [8]:
Xywater = h2o.H2OFrame.from_python(Xy.to_dict('list'), destination_frame='Xy')


In [9]:
Xy_train, Xy_valid = Xywater.split_frame([0.8], ['Xy_train', 'Xy_valid'])

In [10]:
xcolumns = Xywater.columns.copy() 
xcolumns.remove('type')  

In [11]:
from h2o.estimators.gbm import H2OGradientBoostingEstimator 
modelGBM = H2OGradientBoostingEstimator(max_depth=50, ntrees=165)
from h2o.estimators.random_forest import H2ORandomForestEstimator 
modelRF = H2ORandomForestEstimator(max_depth=50, ntrees=165, balance_classes=True)

In [12]:
modelGBM.train(training_frame=Xy_train, validation_frame=Xy_valid, x=xcolumns, y='type')
modelRF.train(training_frame=Xy_train, validation_frame=Xy_valid, x=xcolumns, y='type')

In [13]:
modelGBM

Model Details
H2OGradientBoostingEstimator :  Gradient Boosting Method
Model Key:  GBM_model_python_1471642286902_1731

Model Summary: 


,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
,165.0,165.0,9107986.0,0.0,50.0,23.50909,1.0,12687.0,4351.6665




ModelMetricsBinomial: gbm
** Reported on train data. **

MSE: 0.007681162433891717
RMSE: 0.08764224115055318
LogLoss: 0.02982552096743362
Mean Per-Class Error: 0.013732525682334185
AUC: 0.9991806670641685
Gini: 0.998361334128337

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.5209123264068648: 


,apartments,hotel,Error,Rate
apartments,37526.0,1482.0,0.038,(1482.0/39008.0)
hotel,647.0,169395.0,0.0038,(647.0/170042.0)
Total,38173.0,170877.0,0.0102,(2129.0/209050.0)



Maximum Metrics: Maximum metrics at their respective thresholds



...



Scoring History: 


,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_auc,training_lift,training_classification_error,validation_rmse,validation_logloss,validation_auc,validation_lift,validation_classification_error
,2016-08-20 04:41:31,0.004 sec,0.0,0.3895873,0.4812501,0.5,1.0,0.1865965,0.3882664,0.4788368,0.5,1.0,0.1849574
,2016-08-20 04:41:37,6.292 sec,1.0,0.3576011,0.4103134,0.9851232,1.2294021,0.0546903,0.3586916,0.4131987,0.9566921,1.2146046,0.0696177
,2016-08-20 04:41:44,13.582 sec,2.0,0.3311628,0.3621254,0.9874836,1.2294021,0.0478259,0.3348148,0.3691562,0.9625892,1.1611174,0.0652295
,2016-08-20 04:41:53,22.022 sec,3.0,0.3084052,0.3250297,0.9888026,1.2294021,0.0447022,0.3148243,0.3357410,0.9662810,1.1752200,0.0636198
,2016-08-20 04:42:02,31.728 sec,4.0,0.2884505,0.2947629,0.9898827,1.2294021,0.0418704,0.2977120,0.3087435,0.9690453,1.1846218,0.0610329
---,---,---,---,---,---,---,---,---,---,---,---,---,---
,2016-08-20 04:49:46,8 min 15.582 sec,49.0,0.0938987,0.0394570,0.9988387,1.2294021,0.0107056,0.1790921,0.1132142,0.9858662,1.2269297,0.0422727
,2016-08-20 04:50:41,9 min 10.580 sec,55.0,0.0908795,0.0354610,0.9989769,1.2294021,0.0104568,0.1777161,0.1117559,0.9862450,1.2269297,0.0416786
,2016-08-20 04:51:36,10 min 5.070 sec,62.0,0.0891211,0.0326512,0.9990763,1.2294021,0.0102798,0.1768119,0.1110932,0.9865437,1.2269297,0.0412187
,2016-08-20 04:52:35,11 min 4.570 sec,71.0,0.0879180,0.0304933,0.9991556,1.2294021,0.0101985,0.1759688,0.1108077,0.9868142,1.2269297,0.0408547



See the whole table with table.as_data_frame()

Variable Importances: 


variable,relative_importance,scaled_importance,percentage
Kitchenware,60812.0468750,1.0,0.4123504
Size,9237.3789062,0.1519005,0.0626362
Telephone,8208.8867188,0.1349878,0.0556623
Washing_Machine,4094.0510254,0.0673230,0.0277607
Private_flat_in_building,3083.5512695,0.0507063,0.0209087
---,---,---,---
Sea___Lake___City_view,0.0,0.0,0.0
Sea___Garden___Mountain___River_view,0.0,0.0,0.0
Lake___Garden___Pool___City___River_view,0.0,0.0,0.0
Garden___Pool___City___River_view,0.0,0.0,0.0



See the whole table with table.as_data_frame()


In [14]:
modelRF

Model Details
H2ORandomForestEstimator :  Distributed Random Forest
Model Key:  DRF_model_python_1471642286902_1836

Model Summary: 


,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
,165.0,165.0,34644050.0000000,50.0,50.0,50.0,15662.0,17853.0,16517.51




ModelMetricsBinomial: drf
** Reported on train data. **

MSE: 0.019796910108639373
RMSE: 0.14070149291546047
LogLoss: 0.07930913740887502
Mean Per-Class Error: 0.01970583110269375
AUC: 0.9978150522987355
Gini: 0.9956301045974709

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.6215167233602563: 


,apartments,hotel,Error,Rate
apartments,166209.0,3849.0,0.0226,(3849.0/170058.0)
hotel,2853.0,167189.0,0.0168,(2853.0/170042.0)
Total,169062.0,171038.0,0.0197,(6702.0/340100.0)



Maximum Metrics: Maximum metrics at their respective thresholds



...



Scoring History: 


,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_auc,training_lift,training_classification_error,validation_rmse,validation_logloss,validation_auc,validation_lift,validation_classification_error
,2016-08-20 04:53:30,0.004 sec,0.0,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
,2016-08-20 04:53:38,8.044 sec,1.0,0.2096616,0.9455449,0.9701237,1.2287579,0.0437049,0.2649828,2.1166203,0.9037952,1.1869118,0.0741209
,2016-08-20 04:53:45,14.782 sec,2.0,0.2031399,0.7906457,0.9746515,1.2293940,0.0409426,0.2319134,0.8801287,0.9506857,1.2103230,0.0658810
,2016-08-20 04:53:52,21.509 sec,3.0,0.1957680,0.6625614,0.9785741,1.2294021,0.0378278,0.2219255,0.5486012,0.9645461,1.2167311,0.0585609
,2016-08-20 04:53:58,28.384 sec,4.0,0.1896382,0.5737970,0.9812866,1.2294021,0.0358387,0.2166006,0.3988951,0.9715216,1.2198267,0.0538277
---,---,---,---,---,---,---,---,---,---,---,---,---,---
,2016-08-20 05:07:46,14 min 16.219 sec,120.0,0.1417400,0.0800138,0.9977592,1.2294021,0.0197883,0.2033361,0.1393232,0.9864720,1.2263708,0.0418319
,2016-08-20 05:09:13,15 min 43.254 sec,132.0,0.1413923,0.0797201,0.9977777,1.2294021,0.0197060,0.2032811,0.1393118,0.9864803,1.2263218,0.0419661
,2016-08-20 05:10:49,17 min 18.714 sec,145.0,0.1410808,0.0793648,0.9978020,1.2294021,0.0197118,0.2033875,0.1384586,0.9864889,1.2265105,0.0421769
,2016-08-20 05:12:29,18 min 59.101 sec,159.0,0.1408071,0.0793032,0.9978133,1.2294021,0.0196501,0.2034500,0.1385961,0.9864442,1.2264961,0.0418128



See the whole table with table.as_data_frame()

Variable Importances: 


variable,relative_importance,scaled_importance,percentage
Kitchenware,983785.25,1.0,0.1061926
Stovetop,752343.4375000,0.7647436,0.0812101
Size,606186.9375000,0.6161781,0.0654336
Kitchen,491616.5,0.4997193,0.0530665
Dining_area,460084.4687500,0.4676676,0.0496629
---,---,---,---
Sea___Pool___Mountain___Landmark___City_view,0.0,0.0,0.0
Lake___Mountain___City___River_view,0.0,0.0,0.0
Sea___Lake___Pool___City___River_view,0.0,0.0,0.0
Sea___Garden___Pool___Landmark___City___River_view,0.0,0.0,0.0



See the whole table with table.as_data_frame()


In [23]:
collection = mongo_collection() 
records = collection.find({'$or': [{'type': 'apartments'}, {'type': 'hotel'}]})

In [24]:
train_records, valid_records = split_records(records, 0.8)

ServerSelectionTimeoutError: localhost:27017: [WinError 10061] No connection could be made because the target machine actively refused it

In [20]:
train_features, train_y = rooms_records(train_records) 
valid_features, valid_y = rooms_records(valid_records) 

In [21]:
train_vectorizer = DictVectorizer() 
X_train = train_vectorizer.fit_transform(train_features)

ValueError: Sample sequence X is empty.

In [ ]:
train = pandas.DataFrame(X_train.toarray(), columns=normalize_features(train_vectorizer.get_feature_names())) 
train['type'] = train_y  
X_valid = train_vectorizer.transform(valid_features)
valid = pandas.DataFrame(X_valid.toarray(), columns=normalize_features(train_vectorizer.get_feature_names())) 
valid['type'] = valid_y  

In [ ]:
train_water = h2o.H2OFrame.from_python(train.to_dict('list'), destination_frame='train') 
valid_water = h2o.H2OFrame.from_python(valid.to_dict('list'), destination_frame='valid')

In [ ]:
gbm = H2OGradientBoostingEstimator(max_depth=50, ntrees=165)
rf = H2ORandomForestEstimator(max_depth=50, ntrees=165, balance_classes=True)

In [ ]:
xc = train_water.columns.copy() 
xc.remove('type')  

In [ ]:
gbm.train(training_frame=train_water, validation_frame=valid_water, x=xc, y='type')
rf.train(training_frame=train_water, validation_frame=valid_water, x=xc, y='type')

In [ ]:
gbm

In [ ]:
rf

In [26]:
records.count()

ServerSelectionTimeoutError: localhost:27017: [WinError 10061] No connection could be made because the target machine actively refused it